<a href="https://colab.research.google.com/github/nameshigawa/AE_AnsysBBoxViewer/blob/main/python/ansys_video_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ansys Video Analyzer (Google Colab)

This notebook analyzes a video using YOLO and exports bounding box data as JSON.
The output JSON can be visualized in After Effects using AE_AnsysBBoxViewer.

### 1. Install Required Libraries

First, we need to install the `opencv-python` library for video processing and `ultralytics` which provides the YOLO model for object detection. Run the cell below to install them.

In [ ]:
!pip install opencv-python ultralytics

### 2. Run Script

Run the script, select an MP4 file, and a JSON file containing the analysis results will be downloaded automatically.

In [ ]:
import cv2
import json
from ultralytics import YOLO
from google.colab import files

# Upload video
uploaded = files.upload()
video_path = list(uploaded.keys())[0]

cap = cv2.VideoCapture(video_path)

model = YOLO("yolo11n.pt")

results_json = []
frame_index = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame, verbose=False)

    frame_boxes = []
    for r in results:
        for i, box in enumerate(r.boxes):
            b = box.xyxy[0].tolist()
            conf = float(box.conf[0])
            cls = int(box.cls[0])

            frame_boxes.append({
                "id": i,
                "x": int(b[0]),
                "y": int(b[1]),
                "width": int(b[2] - b[0]),
                "height": int(b[3] - b[1]),
                "label": model.names[cls],
                "conf": round(conf, 3)
            })

    results_json.append(frame_boxes)
    frame_index += 1

cap.release()

# Save JSON
json_path = video_path.replace(".mp4", "_bbox.json")
with open(json_path, "w") as f:
    json.dump(results_json, f, indent=2)

# Download JSON
files.download(json_path)
